ЗАДАЧИ ДЛЯ ПРАКТИКИ ООП

**1. БАЗОВЫЕ ФИНАНСОВЫЕ ИНСТРУМЕНТЫ:**
- Реализуйте класс `EuropeanPutOption`, наследующий от `Option`
- Реализуйте класс `AmericanCallOption` с возможностью досрочного исполнения
- Добавьте валидацию даты истечения в базовый класс `Option`
- Создайте класс `Stock` (акция) как наследник `AbstractAsset`

In [ ]:
from datetime import date, timedelta

# Базовый класс для всех активов
class Asset:
    def __init__(self, symbol: str, price: float):
        self.symbol = symbol
        self.price = price
    
    def __str__(self):
        return f"{self.symbol}: ${self.price:.2f}"

# Класс акции
class Stock(Asset):
    def __init__(self, symbol: str, price: float):
        super().__init__(symbol, price)

# Базовый класс опциона
class Option(Asset):
    def __init__(self, symbol: str, stock: Stock, strike: float, expiration: date):
        super().__init__(symbol, stock.price)
        self.stock = stock
        self.strike = strike
        self.expiration = expiration
        
        # Проверяем дату
        if expiration <= date.today():
            raise ValueError("Дата экспирации должна быть в будущем")
    
    def days_left(self):
        return (self.expiration - date.today()).days
    
    def is_expired(self):
        return self.days_left() <= 0
    
    def intrinsic_value(self):
        """Базовая стоимость опциона"""
        return max(0, self.stock.price - self.strike)

# Европейский пут-опцион
class EuropeanPutOption(Option):
    def __init__(self, symbol: str, stock: Stock, strike: float, expiration: date):
        super().__init__(symbol, stock, strike, expiration)
    
    def intrinsic_value(self):
        # Для пут-опциона: strike - price
        return max(0, self.strike - self.stock.price)
    
    def can_exercise(self):
        # Европейский - только в дату экспирации
        return date.today() == self.expiration
    
    def get_value(self):
        if self.is_expired():
            return self.intrinsic_value()
        # Простой расчет стоимости
        return self.intrinsic_value() + (self.days_left() * 0.1)

# Американский кол-опцион
class AmericanCallOption(Option):
    def __init__(self, symbol: str, stock: Stock, strike: float, expiration: date):
        super().__init__(symbol, stock, strike, expiration)
    
    def can_exercise(self):
        # Американский - в любой день
        return not self.is_expired() and self.intrinsic_value() > 0
    
    def exercise(self):
        if not self.can_exercise():
            raise ValueError("Нельзя исполнить опцион")
        return self.intrinsic_value()
    
    def get_value(self):
        if self.is_expired():
            return self.intrinsic_value()
        # Американский опцион всегда стоит не меньше intrinsic value
        return max(self.intrinsic_value(), self.intrinsic_value() + (self.days_left() * 0.1))

# Демонстрация
if __name__ == "__main__":
    print("=== ПРОСТАЯ СИСТЕМА ФИНАНСОВЫХ ИНСТРУМЕНТОВ ===\n")
    
    # Создаем акцию
    apple = Stock("AAPL", 150.0)
    print(f"📈 Акция: {apple}")
    
    # Европейский пут-опцион
    exp_date = date.today() + timedelta(days=30)
    put_option = EuropeanPutOption("AAPL_PUT", apple, 140.0, exp_date)
    
    print(f"\n🇪🇺 Европейский пут-опцион:")
    print(f"   Страйк: ${put_option.strike:.2f}")
    print(f"   Дней до экспирации: {put_option.days_left()}")
    print(f"   Внутренняя стоимость: ${put_option.intrinsic_value():.2f}")
    print(f"   Полная стоимость: ${put_option.get_value():.2f}")
    print(f"   Можно исполнить: {put_option.can_exercise()}")
    
    # Американский кол-опцион
    call_option = AmericanCallOption("AAPL_CALL", apple, 160.0, exp_date)
    
    print(f"\n🇺🇸 Американский кол-опцион:")
    print(f"   Страйк: ${call_option.strike:.2f}")
    print(f"   Внутренняя стоимость: ${call_option.intrinsic_value():.2f}")
    print(f"   Полная стоимость: ${call_option.get_value():.2f}")
    print(f"   Можно исполнить: {call_option.can_exercise()}")
    
    # Пробуем исполнить
    try:
        profit = call_option.exercise()
        print(f"   💰 Исполнение: ${profit:.2f}")
    except ValueError as e:
        print(f"   ❌ Нельзя исполнить: {e}")
    
    # Меняем цену акции
    print(f"\n🔄 Меняем цену AAPL на $170...")
    apple.price = 170.0
    
    print(f"   Новая цена акции: ${apple.price:.2f}")
    print(f"   Пут стоимость: ${put_option.get_value():.2f}")
    print(f"   Кол стоимость: ${call_option.get_value():.2f}")
    print(f"   Кол можно исполнить: {call_option.can_exercise()}")

**2. РАСШИРЕННЫЕ ОПЦИОНЫ:**
- Реализуйте класс `BinaryOption` (бинарный опцион)
- Создайте класс `BarrierOption` (барьерный опцион)
- Добавьте метод расчета Greeks (дельта, гамма, тета, вега) для опционов


In [ ]:
from datetime import date, timedelta
import math

# Базовый класс опциона
class Option:
    def __init__(self, symbol: str, stock_price: float, strike: float, expiration: date):
        self.symbol = symbol
        self.stock_price = stock_price
        self.strike = strike
        self.expiration = expiration
        self.days_left = (expiration - date.today()).days
    
    def get_value(self):
        return 0
    
    def get_greeks(self):
        return {"delta": 0, "gamma": 0, "theta": 0, "vega": 0}

# Бинарный опцион (выплата фиксированная)
class BinaryOption(Option):
    def __init__(self, symbol: str, stock_price: float, strike: float, 
                 expiration: date, payout: float = 100):
        super().__init__(symbol, stock_price, strike, expiration)
        self.payout = payout
    
    def get_value(self):
        """Стоимость: либо полная выплата, либо 0"""
        if self.stock_price > self.strike:
            return self.payout
        return 0
    
    def get_greeks(self):
        """Упрощенные греки для бинарного опциона"""
        return {
            "delta": 0.5,  # Примерное значение
            "gamma": 0.1,
            "theta": -0.05 * self.days_left,
            "vega": 0.02
        }

# Барьерный опцион (активируется/деактивируется при достижении барьера)
class BarrierOption(Option):
    def __init__(self, symbol: str, stock_price: float, strike: float, 
                 expiration: date, barrier: float, option_type: str = "up_and_out"):
        super().__init__(symbol, stock_price, strike, expiration)
        self.barrier = barrier
        self.option_type = option_type  # "up_and_out", "down_and_out", etc.
    
    def is_active(self):
        """Проверка, активен ли опцион"""
        if self.option_type == "up_and_out":
            return self.stock_price < self.barrier
        elif self.option_type == "down_and_out":
            return self.stock_price > self.barrier
        return True
    
    def get_value(self):
        if not self.is_active():
            return 0
        
        # Базовая стоимость как у обычного опциона
        if self.option_type in ["up_and_out", "down_and_out"]:
            return max(0, self.stock_price - self.strike) * 0.8  # Скидка за барьер
        return max(0, self.stock_price - self.strike)
    
    def get_greeks(self):
        """Греки с учетом барьера"""
        base_delta = 0.6 if self.stock_price > self.strike else 0.4
        
        if not self.is_active():
            return {"delta": 0, "gamma": 0, "theta": 0, "vega": 0}
        
        return {
            "delta": base_delta * 0.9,  # Чуть меньше из-за барьера
            "gamma": 0.08,
            "theta": -0.04 * self.days_left,
            "vega": 0.015
        }

# Обычный опцион с расчетом греков
class VanillaOption(Option):
    def __init__(self, symbol: str, stock_price: float, strike: float, 
                 expiration: date, option_type: str = "call"):
        super().__init__(symbol, stock_price, strike, expiration)
        self.option_type = option_type
    
    def get_value(self):
        if self.option_type == "call":
            return max(0, self.stock_price - self.strike)
        else:  # put
            return max(0, self.strike - self.stock_price)
    
    def get_greeks(self):
        """Упрощенный расчет греков"""
        # Дельта: чувствительность к цене акции
        if self.option_type == "call":
            delta = 0.7 if self.stock_price > self.strike else 0.3
        else:  # put
            delta = -0.3 if self.stock_price < self.strike else -0.7
        
        # Гамма: скорость изменения дельты
        gamma = 0.1
        
        # Тета: временное decay (стоимость уменьшается со временем)
        theta = -0.1 * (30 - self.days_left)
        
        # Вега: чувствительность к волатильности
        vega = 0.15
        
        return {
            "delta": delta,
            "gamma": gamma,
            "theta": theta,
            "vega": vega
        }

# Демонстрация
if __name__ == "__main__":
    print("=== РАСШИРЕННЫЕ ОПЦИОНЫ И ГРЕКИ ===\n")
    
    stock_price = 150.0
    expiration = date.today() + timedelta(days=30)
    
    # Бинарный опцион
    binary = BinaryOption("BINARY_AAPL", stock_price, 155.0, expiration, payout=100)
    print(f"📊 Бинарный опцион:")
    print(f"   Стоимость: ${binary.get_value():.2f}")
    print(f"   Выплата: ${binary.payout:.2f}")
    print(f"   Греки: {binary.get_greeks()}")
    
    # Барьерный опцион
    barrier = BarrierOption("BARRIER_AAPL", stock_price, 145.0, expiration, 
                           barrier=160.0, option_type="up_and_out")
    print(f"\n🚧 Барьерный опцион (up&out @ ${barrier.barrier:.2f}):")
    print(f"   Активен: {barrier.is_active()}")
    print(f"   Стоимость: ${barrier.get_value():.2f}")
    print(f"   Греки: {barrier.get_greeks()}")
    
    # Обычный опцион
    vanilla_call = VanillaOption("VANILLA_CALL", stock_price, 148.0, expiration, "call")
    vanilla_put = VanillaOption("VANILLA_PUT", stock_price, 152.0, expiration, "put")
    
    print(f"\n🍦 Обычный кол-опцион:")
    print(f"   Стоимость: ${vanilla_call.get_value():.2f}")
    print(f"   Греки: {vanilla_call.get_greeks()}")
    
    print(f"\n🍦 Обычный пут-опцион:")
    print(f"   Стоимость: ${vanilla_put.get_value():.2f}")
    print(f"   Греки: {vanilla_put.get_greeks()}")
    
    print(f"\n📈 Изменяем цену акции на $160...")
    # Обновляем цены для всех опционов
    for option in [binary, barrier, vanilla_call, vanilla_put]:
        option.stock_price = 160.0
        option.days_left = (expiration - date.today()).days
        
        print(f"\n{option.symbol}:")
        print(f"   Новая стоимость: ${option.get_value():.2f}")
        print(f"   Новые греки: {option.get_greeks()}")
        if hasattr(option, 'is_active'):
            print(f"   Активен: {option.is_active()}")

**3. ПОРТФЕЛЬ И УПРАВЛЕНИЕ АКТИВАМИ:**
- Реализуйте метод `__repr__` для класса `Portfolio`
- Добавьте метод `__getitem__` для доступа к активам по индексу
- Реализуйте метод `__contains__` для проверки наличия актива в портфеле
- Добавьте метод `remove_asset` для удаления актива из портфеля
- Создайте метод `calculate_portfolio_risk` для расчета риска портфеля


In [ ]:
class Portfolio:
    def __init__(self, name: str):
        self.name = name
        self.assets = []  # Список активов
        self.weights = {}  # Веса активов
    
    def add_asset(self, asset, weight: float = 1.0):
        """Добавить актив в портфель"""
        self.assets.append(asset)
        self.weights[asset.symbol] = weight
    
    def remove_asset(self, symbol: str):
        """Удалить актив из портфеля по символу"""
        self.assets = [asset for asset in self.assets if asset.symbol != symbol]
        if symbol in self.weights:
            del self.weights[symbol]
    
    def __repr__(self):
        """Представление для разработчика"""
        assets_str = ", ".join([f"{asset.symbol}(${asset.price})" for asset in self.assets])
        return f"Portfolio(name='{self.name}', assets=[{assets_str}], total_value=${self.total_value():.2f})"
    
    def __str__(self):
        """Строковое представление для пользователя"""
        return f"Портфель '{self.name}': {len(self.assets)} активов, стоимость: ${self.total_value():.2f}"
    
    def __getitem__(self, index):
        """Доступ к активам по индексу или срезу"""
        return self.assets[index]
    
    def __contains__(self, asset):
        """Проверка наличия актива в портфеле"""
        if hasattr(asset, 'symbol'):
            return any(a.symbol == asset.symbol for a in self.assets)
        elif isinstance(asset, str):
            return any(a.symbol == asset for a in self.assets)
        return False
    
    def __len__(self):
        """Количество активов в портфеле"""
        return len(self.assets)
    
    def total_value(self):
        """Общая стоимость портфеля"""
        return sum(asset.price for asset in self.assets)
    
    def calculate_portfolio_risk(self):
        """Упрощенный расчет риска портфеля"""
        if not self.assets:
            return 0
        
        # Простой расчет: средняя волатильность активов
        total_risk = 0
        for asset in self.assets:
            # Предположим, что риск зависит от цены и типа актива
            if hasattr(asset, 'volatility'):
                risk = asset.volatility
            else:
                # Простая эвристика: дорогие активы более рискованные
                risk = min(asset.price * 0.1, 50)  # Максимум 50% риска
            
            weight = self.weights.get(asset.symbol, 1.0)
            total_risk += risk * weight
        
        return total_risk / len(self.assets)
    
    def get_asset_by_symbol(self, symbol: str):
        """Найти актив по символу"""
        for asset in self.assets:
            if asset.symbol == symbol:
                return asset
        return None
    
    def portfolio_info(self):
        """Информация о портфеле"""
        info = f"=== ПОРТФЕЛЬ: {self.name} ===\n"
        info += f"Активов: {len(self.assets)}\n"
        info += f"Общая стоимость: ${self.total_value():.2f}\n"
        info += f"Уровень риска: {self.calculate_portfolio_risk():.1f}%\n"
        info += "Активы:\n"
        
        for i, asset in enumerate(self.assets, 1):
            weight = self.weights.get(asset.symbol, 1.0)
            info += f"  {i}. {asset.symbol}: ${asset.price:.2f} (вес: {weight})\n"
        
        return info

# Базовый класс актива
class Asset:
    def __init__(self, symbol: str, price: float, volatility: float = 10.0):
        self.symbol = symbol
        self.price = price
        self.volatility = volatility  # Волатильность в %
    
    def __repr__(self):
        return f"Asset('{self.symbol}', ${self.price:.2f})"

# Класс акции
class Stock(Asset):
    def __init__(self, symbol: str, price: float, volatility: float = 15.0):
        super().__init__(symbol, price, volatility)

# Класс облигации
class Bond(Asset):
    def __init__(self, symbol: str, price: float, volatility: float = 5.0):
        super().__init__(symbol, price, volatility)

# Демонстрация
if __name__ == "__main__":
    print("=== СИСТЕМА УПРАВЛЕНИЯ ПОРТФЕЛЕМ ===\n")
    
    # Создаем активы
    apple = Stock("AAPL", 150.0, 20.0)
    google = Stock("GOOGL", 2800.0, 25.0)
    microsoft = Stock("MSFT", 300.0, 18.0)
    treasury_bond = Bond("TBOND", 100.0, 3.0)
    
    # Создаем портфель
    my_portfolio = Portfolio("Мой инвестиционный портфель")
    
    # Добавляем активы с весами
    my_portfolio.add_asset(apple, weight=0.3)
    my_portfolio.add_asset(google, weight=0.4)
    my_portfolio.add_asset(microsoft, weight=0.2)
    my_portfolio.add_asset(treasury_bond, weight=0.1)
    
    # Тестируем методы
    print("1. 📊 Информация о портфеле:")
    print(my_portfolio.portfolio_info())
    
    print("\n2. 🔍 Проверка методов доступа:")
    print(f"   repr: {repr(my_portfolio)}")
    print(f"   str: {my_portfolio}")
    print(f"   Количество активов: {len(my_portfolio)}")
    print(f"   Первый актив: {my_portfolio[0]}")
    print(f"   Последний актив: {my_portfolio[-1]}")
    print(f"   Есть ли AAPL в портфеле: {apple in my_portfolio}")
    print(f"   Есть ли TSLA в портфеле: {'TSLA' in my_portfolio}")
    
    print("\n3. 🗑️ Тестируем удаление актива:")
    print(f"   До удаления: {len(my_portfolio)} активов")
    my_portfolio.remove_asset("GOOGL")
    print(f"   После удаления GOOGL: {len(my_portfolio)} активов")
    print(f"   Есть ли GOOGL теперь: {'GOOGL' in my_portfolio}")
    
    print("\n4. 📈 Обновленная информация:")
    print(my_portfolio.portfolio_info())
    
    print("\n5. 🔄 Итерация по портфелю:")
    for i, asset in enumerate(my_portfolio):
        print(f"   {i+1}. {asset.symbol} - ${asset.price:.2f}")
    
    print("\n6. 🎯 Поиск актива:")
    found_asset = my_portfolio.get_asset_by_symbol("AAPL")
    if found_asset:
        print(f"   Найден: {found_asset}")
    else:
        print("   Актив не найден")

**5. ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ:**
- Добавьте `__eq__`, `__lt__`, `__le__` для сравнения портфелей
- Реализуйте `__mul__` для умножения портфеля на число (масштабирование)
- Добавьте `__str__` для красивого вывода портфеля
- Реализуйте `__bool__` для проверки, является ли портфель пустым


In [ ]:
class Portfolio:
    def __init__(self, name: str):
        self.name = name
        self.assets = []
        self.weights = {}
    
    def add_asset(self, asset, weight: float = 1.0):
        """Добавить актив в портфель"""
        self.assets.append(asset)
        self.weights[asset.symbol] = weight
    
    def remove_asset(self, symbol: str):
        """Удалить актив из портфеля"""
        self.assets = [asset for asset in self.assets if asset.symbol != symbol]
        if symbol in self.weights:
            del self.weights[symbol]
    
    def total_value(self):
        """Общая стоимость портфеля"""
        return sum(asset.price for asset in self.assets)
    
    # ===== DUNDER МЕТОДЫ =====
    
    def __repr__(self):
        """Для разработчиков"""
        return f"Portfolio('{self.name}', assets={len(self.assets)}, value=${self.total_value():.2f})"
    
    def __str__(self):
        """Красивый вывод для пользователей"""
        if not self.assets:
            return f"📭 Портфель '{self.name}' (пустой)"
        
        assets_list = "\n".join([f"   📈 {asset.symbol}: ${asset.price:.2f}" for asset in self.assets])
        return f"📊 Портфель '{self.name}'\n{assets_list}\n   💰 Итого: ${self.total_value():.2f}"
    
    def __bool__(self):
        """Проверка, не пустой ли портфель"""
        return len(self.assets) > 0
    
    def __eq__(self, other):
        """Сравнение по стоимости портфелей"""
        if not isinstance(other, Portfolio):
            return False
        return self.total_value() == other.total_value()
    
    def __lt__(self, other):
        """Сравнение: меньше ли стоимость"""
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self.total_value() < other.total_value()
    
    def __le__(self, other):
        """Сравнение: меньше или равно"""
        if not isinstance(other, Portfolio):
            return NotImplemented
        return self.total_value() <= other.total_value()
    
    def __mul__(self, factor):
        """Умножение портфеля на число (масштабирование цен)"""
        if not isinstance(factor, (int, float)):
            return NotImplemented
        
        # Создаем новый портфель с масштабированными ценами
        new_portfolio = Portfolio(f"{self.name} x{factor}")
        
        for asset in self.assets:
            # Создаем новый актив с измененной ценой
            scaled_asset = Asset(
                asset.symbol, 
                asset.price * factor,
                getattr(asset, 'volatility', 10.0)
            )
            weight = self.weights.get(asset.symbol, 1.0)
            new_portfolio.add_asset(scaled_asset, weight)
        
        return new_portfolio
    
    def __rmul__(self, factor):
        """Умножение числа на портфель"""
        return self.__mul__(factor)
    
    def __add__(self, other):
        """Объединение двух портфелей"""
        if not isinstance(other, Portfolio):
            return NotImplemented
        
        new_portfolio = Portfolio(f"{self.name} + {other.name}")
        
        # Добавляем активы из первого портфеля
        for asset in self.assets:
            weight = self.weights.get(asset.symbol, 1.0)
            new_portfolio.add_asset(asset, weight)
        
        # Добавляем активы из второго портфеля
        for asset in other.assets:
            weight = other.weights.get(asset.symbol, 1.0)
            # Если актив уже есть, увеличиваем вес
            if asset.symbol in new_portfolio.weights:
                new_portfolio.weights[asset.symbol] += weight
            else:
                new_portfolio.add_asset(asset, weight)
        
        return new_portfolio
    
    def __getitem__(self, index):
        """Доступ к активам по индексу"""
        return self.assets[index]
    
    def __contains__(self, asset):
        """Проверка наличия актива"""
        if hasattr(asset, 'symbol'):
            return any(a.symbol == asset.symbol for a in self.assets)
        return any(a.symbol == asset for a in self.assets)
    
    def __len__(self):
        """Количество активов"""
        return len(self.assets)
    
    def __iter__(self):
        """Итерация по активам"""
        return iter(self.assets)

# Базовый класс актива
class Asset:
    def __init__(self, symbol: str, price: float, volatility: float = 10.0):
        self.symbol = symbol
        self.price = price
        self.volatility = volatility
    
    def __repr__(self):
        return f"Asset('{self.symbol}', ${self.price:.2f})"

# Демонстрация
if __name__ == "__main__":
    print("=== ДОПОЛНИТЕЛЬНЫЕ DUNDER МЕТОДЫ ===\n")
    
    # Создаем активы
    apple = Asset("AAPL", 150.0)
    google = Asset("GOOGL", 2800.0)
    microsoft = Asset("MSFT", 300.0)
    
    # Создаем портфели
    portfolio1 = Portfolio("Технологический портфель")
    portfolio1.add_asset(apple)
    portfolio1.add_asset(google)
    
    portfolio2 = Portfolio("Софтверный портфель") 
    portfolio2.add_asset(microsoft)
    portfolio2.add_asset(Asset("ORCL", 80.0))
    
    empty_portfolio = Portfolio("Пустой портфель")
    
    print("1. 📋 ИНФОРМАЦИЯ О ПОРТФЕЛЯХ:")
    print(f"   Portfolio 1: {portfolio1}")
    print(f"   Portfolio 2: {portfolio2}")
    print(f"   Empty: {empty_portfolio}")
    
    print("\n2. 🔍 ПРОВЕРКА BOOL:")
    print(f"   Portfolio1 не пустой: {bool(portfolio1)}")
    print(f"   Empty портфель не пустой: {bool(empty_portfolio)}")
    
    print("\n3. ⚖️ СРАВНЕНИЕ ПОРТФЕЛЕЙ:")
    print(f"   Portfolio1 == Portfolio2: {portfolio1 == portfolio2}")
    print(f"   Portfolio1 < Portfolio2: {portfolio1 < portfolio2}")
    print(f"   Portfolio1 <= Portfolio2: {portfolio1 <= portfolio2}")
    print(f"   Portfolio1 > Portfolio2: {portfolio1 > portfolio2}")  # Автоматически из < и ==
    
    print("\n4. 🔢 УМНОЖЕНИЕ ПОРТФЕЛЯ (МАСШТАБИРОВАНИЕ):")
    doubled_portfolio = portfolio1 * 2
    print(f"   Исходный: {portfolio1}")
    print(f"   Удвоенный: {doubled_portfolio}")
    
    half_portfolio = 0.5 * portfolio1  # Проверяем __rmul__
    print(f"   Половинный: {half_portfolio}")
    
    print("\n5. ➕ СЛОЖЕНИЕ ПОРТФЕЛЕЙ:")
    combined = portfolio1 + portfolio2
    print(f"   Объединенный: {combined}")
    
    print("\n6. 🎯 ИТЕРАЦИЯ И ДОСТУП:")
    print(f"   Активы в portfolio1:")
    for asset in portfolio1:
        print(f"     - {asset.symbol}: ${asset.price:.2f}")
    
    print(f"   Первый актив: {portfolio1[0]}")
    print(f"   Последний актив: {portfolio1[-1]}")
    print(f"   Есть ли AAPL: {apple in portfolio1}")
    print(f"   Есть ли TSLA: {'TSLA' in portfolio1}")
    print(f"   Количество активов: {len(portfolio1)}")
    
    print("\n7. 🧪 ДОПОЛНИТЕЛЬНЫЕ ПРОВЕРКИ:")
    # Создаем портфель с такой же стоимостью для проверки равенства
    equal_value_portfolio = Portfolio("Равный по стоимости")
    equal_value_portfolio.add_asset(Asset("TEST", portfolio1.total_value()))
    
    print(f"   Стоимость portfolio1: ${portfolio1.total_value():.2f}")
    print(f"   Стоимость equal_value: ${equal_value_portfolio.total_value():.2f}")
    print(f"   portfolio1 == equal_value: {portfolio1 == equal_value_portfolio}")

**6. ПРОДВИНУТЫЕ ЗАДАЧИ:**
- Создайте декоратор `@validate_option_params` для валидации параметров опционов
- Реализуйте класс `PortfolioManager` с методами для управления несколькими портфелями
- Добавьте поддержку сериализации/десериализации портфелей (pickle)
- Создайте контекстный менеджер для временного изменения параметров портфеля

In [ ]:
import pickle
from datetime import date, timedelta
from contextlib import contextmanager
from functools import wraps
from typing import Dict, List, Any

# ===== ДЕКОРАТОР ВАЛИДАЦИИ =====
def validate_option_params(func):
    """Декоратор для валидации параметров опционов"""
    @wraps(func)
    def wrapper(self, symbol, stock_price, strike, expiration, *args, **kwargs):
        # Валидация типов
        if not isinstance(symbol, str) or not symbol:
            raise ValueError("Symbol должен быть непустой строкой")
        
        if not isinstance(stock_price, (int, float)) or stock_price <= 0:
            raise ValueError("Цена акции должна быть положительным числом")
        
        if not isinstance(strike, (int, float)) or strike <= 0:
            raise ValueError("Страйк должен быть положительным числом")
        
        if not isinstance(expiration, date) or expiration <= date.today():
            raise ValueError("Дата экспирации должна быть в будущем")
        
        return func(self, symbol, stock_price, strike, expiration, *args, **kwargs)
    return wrapper

# ===== БАЗОВЫЕ КЛАССЫ =====
class Asset:
    def __init__(self, symbol: str, price: float):
        self.symbol = symbol
        self.price = price
    
    def __repr__(self):
        return f"Asset('{self.symbol}', ${self.price:.2f})"

class Option(Asset):
    @validate_option_params
    def __init__(self, symbol: str, stock_price: float, strike: float, expiration: date):
        super().__init__(symbol, stock_price)
        self.strike = strike
        self.expiration = expiration
    
    def get_value(self):
        return max(0, self.price - self.strike)
    
    def __repr__(self):
        return f"Option('{self.symbol}', strike=${self.strike}, exp={self.expiration})"

class Portfolio:
    def __init__(self, name: str):
        self.name = name
        self.assets = []
    
    def add_asset(self, asset):
        self.assets.append(asset)
    
    def remove_asset(self, symbol):
        self.assets = [a for a in self.assets if a.symbol != symbol]
    
    def total_value(self):
        return sum(a.price for a in self.assets)
    
    def __repr__(self):
        return f"Portfolio('{self.name}', assets={len(self.assets)}, value=${self.total_value():.2f})"
    
    def __getstate__(self):
        """Для сериализации"""
        return {
            'name': self.name,
            'assets': self.assets
        }
    
    def __setstate__(self, state):
        """Для десериализации"""
        self.name = state['name']
        self.assets = state['assets']

# ===== МЕНЕДЖЕР ПОРТФЕЛЕЙ =====
class PortfolioManager:
    """Класс для управления несколькими портфелями"""
    
    def __init__(self):
        self.portfolios = {}
    
    def create_portfolio(self, name: str) -> Portfolio:
        """Создать новый портфель"""
        if name in self.portfolios:
            raise ValueError(f"Портфель с именем '{name}' уже существует")
        
        portfolio = Portfolio(name)
        self.portfolios[name] = portfolio
        return portfolio
    
    def get_portfolio(self, name: str) -> Portfolio:
        """Получить портфель по имени"""
        if name not in self.portfolios:
            raise ValueError(f"Портфель '{name}' не найден")
        return self.portfolios[name]
    
    def delete_portfolio(self, name: str):
        """Удалить портфель"""
        if name not in self.portfolios:
            raise ValueError(f"Портфель '{name}' не найден")
        del self.portfolios[name]
    
    def list_portfolios(self) -> List[str]:
        """Список всех портфелей"""
        return list(self.portfolios.keys())
    
    def total_assets_value(self) -> float:
        """Общая стоимость всех портфелей"""
        return sum(portfolio.total_value() for portfolio in self.portfolios.values())
    
    def get_portfolio_stats(self) -> Dict[str, Any]:
        """Статистика по всем портфелям"""
        return {
            'total_portfolios': len(self.portfolios),
            'total_assets': sum(len(p.assets) for p in self.portfolios.values()),
            'total_value': self.total_assets_value(),
            'average_value': self.total_assets_value() / len(self.portfolios) if self.portfolios else 0
        }
    
    def save_to_file(self, filename: str):
        """Сохранить все портфели в файл"""
        with open(filename, 'wb') as f:
            pickle.dump(self.portfolios, f)
        print(f"💾 Портфели сохранены в {filename}")
    
    def load_from_file(self, filename: str):
        """Загрузить портфели из файла"""
        try:
            with open(filename, 'rb') as f:
                self.portfolios = pickle.load(f)
            print(f"📂 Портфели загружены из {filename}")
        except FileNotFoundError:
            print(f"❌ Файл {filename} не найден")
        except Exception as e:
            print(f"❌ Ошибка загрузки: {e}")
    
    def __repr__(self):
        return f"PortfolioManager(portfolios={len(self.portfolios)})"

# ===== КОНТЕКСТНЫЙ МЕНЕДЖЕР =====
@contextmanager
def temporary_portfolio_changes(portfolio: Portfolio, temp_assets: List[Asset] = None):
    """
    Контекстный менеджер для временного изменения портфеля
    """
    original_assets = portfolio.assets.copy()
    
    try:
        # Временно заменяем активы
        if temp_assets is not None:
            portfolio.assets = temp_assets
        
        print(f"🔄 Временные изменения применены к портфелю '{portfolio.name}'")
        yield portfolio
        
    finally:
        # Всегда восстанавливаем оригинальное состояние
        portfolio.assets = original_assets
        print(f"🔄 Оригинальное состояние портфеля '{portfolio.name}' восстановлено")

# ===== ДЕМОНСТРАЦИЯ =====
def main():
    print("=== ПРОДВИНУТЫЕ ВОЗМОЖНОСТИ СИСТЕМЫ ===\n")
    
    # Создаем менеджер портфелей
    manager = PortfolioManager()
    
    print("1. 🎯 ДЕКОРАТОР ВАЛИДАЦИИ:")
    try:
        # Правильные параметры
        valid_option = Option("AAPL_CALL", 150.0, 140.0, date.today() + timedelta(days=30))
        print(f"   ✅ Создан опцион: {valid_option}")
        
        # Неправильные параметры
        invalid_option = Option("", 150.0, 140.0, date.today() + timedelta(days=30))
    except ValueError as e:
        print(f"   ✅ Поймана ошибка валидации: {e}")
    
    print("\n2. 📊 МЕНЕДЖЕР ПОРТФЕЛЕЙ:")
    
    # Создаем портфели
    tech_portfolio = manager.create_portfolio("Технологии")
    tech_portfolio.add_asset(Asset("AAPL", 150.0))
    tech_portfolio.add_asset(Asset("GOOGL", 2800.0))
    
    energy_portfolio = manager.create_portfolio("Энергетика")
    energy_portfolio.add_asset(Asset("XOM", 80.0))
    energy_portfolio.add_asset(Asset("CVX", 120.0))
    
    print(f"   Портфели: {manager.list_portfolios()}")
    print(f"   Статистика: {manager.get_portfolio_stats()}")
    
    print("\n3. 💾 СЕРИАЛИЗАЦИЯ ПОРТФЕЛЕЙ:")
    
    # Сохраняем портфели
    manager.save_to_file("portfolios.pkl")
    
    # Создаем нового менеджера и загружаем
    new_manager = PortfolioManager()
    new_manager.load_from_file("portfolios.pkl")
    
    print(f"   Загруженные портфели: {new_manager.list_portfolios()}")
    print(f"   Статистика после загрузки: {new_manager.get_portfolio_stats()}")
    
    print("\n4. 🔄 КОНТЕКСТНЫЙ МЕНЕДЖЕР:")
    
    # Создаем временные активы
    temp_assets = [
        Asset("TSLA", 250.0),
        Asset("NVDA", 500.0)
    ]
    
    print(f"   Исходный портфель: {tech_portfolio}")
    
    # Используем контекстный менеджер
    with temporary_portfolio_changes(tech_portfolio, temp_assets) as temp_portfolio:
        print(f"   Временный портфель: {temp_portfolio}")
        print(f"   Стоимость временного: ${temp_portfolio.total_value():.2f}")
    
    print(f"   Восстановленный портфель: {tech_portfolio}")
    
    print("\n5. 🧪 ДОПОЛНИТЕЛЬНЫЕ ТЕСТЫ:")
    
    # Тест удаления портфеля
    print(f"   До удаления: {manager.list_portfolios()}")
    manager.delete_portfolio("Энергетика")
    print(f"   После удаления: {manager.list_portfolios()}")
    
    # Тест ошибок
    try:
        manager.get_portfolio("Несуществующий")
    except ValueError as e:
        print(f"   ✅ Обработана ошибка: {e}")
    
    # Контекстный менеджер без изменений активов
    with temporary_portfolio_changes(tech_portfolio) as unchanged_portfolio:
        print(f"   Портфель без изменений активов: {unchanged_portfolio}")

if __name__ == "__main__":
    main()

# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 5: Alpha стратегии
# Реализуйте систему альфа-стратегий для портфеля

from abc import ABC, abstractmethod
from typing import List, Dict, Any
import numpy as np

# TODO: Реализуйте абстрактный класс Alpha
class Alpha(ABC):
    def __init__(self, name: str = 'Alpha'):
        # TODO: Реализуйте конструктор
        pass
    
    @abstractmethod
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        # TODO: Реализуйте абстрактный метод расчета альфы
        pass
    
    def __repr__(self) -> str:
        # TODO: Реализуйте вывод
        pass

# TODO: Реализуйте класс MomentumAlpha
class MomentumAlpha(Alpha):
    def __init__(self, lookback_period: int = 20, name: str = 'MomentumAlpha'):
        # TODO: Реализуйте конструктор
        pass
    
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет альфы на основе моментума
        # Альфа = средняя доходность за период - бета * средняя рыночная доходность
        pass

# TODO: Реализуйте класс MeanReversionAlpha
class MeanReversionAlpha(Alpha):
    def __init__(self, reversion_threshold: float = 0.02, name: str = 'MeanReversionAlpha'):
        # TODO: Реализуйте конструктор
        pass
    
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет альфы на основе возврата к среднему
        pass

# TODO: Реализуйте класс AlphaPortfolio
class AlphaPortfolio(Portfolio):
    def __init__(
        self,
        assets: List[AbstractAsset] = None,
        alpha_strategy: Alpha = None,
        name: str = 'AlphaPortfolio'
    ):
        # TODO: Реализуйте конструктор
        pass
    
    def calculate_portfolio_alpha(self, market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет альфы портфеля
        pass
    
    def calculate_beta(self, market_returns: List[float]) -> float:
        # TODO: Реализуйте расчет беты портфеля
        pass
    
    def get_risk_metrics(self, market_returns: List[float]) -> Dict[str, float]:
        # TODO: Реализуйте расчет различных метрик риска
        # Возвращайте словарь с ключами: 'alpha', 'beta', 'sharpe_ratio', 'volatility'
        pass


In [ ]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any
import numpy as np

# Базовый класс актива для совместимости
class AbstractAsset:
    def __init__(self, symbol: str, price: float):
        self.symbol = symbol
        self.price = price
        self.returns_history = []

# Базовый класс портфеля для наследования
class Portfolio:
    def __init__(self, assets: List[AbstractAsset] = None):
        self.assets = assets if assets is not None else []
    
    def total_value(self):
        return sum(asset.price for asset in self.assets)

# TODO: Реализуйте абстрактный класс Alpha
class Alpha(ABC):
    def __init__(self, name: str = 'Alpha'):
        self.name = name
    
    @abstractmethod
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        """Расчет альфы - избыточной доходности относительно рынка"""
        pass
    
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}('{self.name}')"

# TODO: Реализуйте класс MomentumAlpha
class MomentumAlpha(Alpha):
    def __init__(self, lookback_period: int = 20, name: str = 'MomentumAlpha'):
        super().__init__(name)
        self.lookback_period = lookback_period
    
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        """Альфа на основе моментума: тренд продолжается"""
        if len(returns) < self.lookback_period or len(market_returns) < self.lookback_period:
            return 0.0
        
        # Берем последние данные за период
        recent_returns = returns[-self.lookback_period:]
        recent_market_returns = market_returns[-self.lookback_period:]
        
        # Расчет беты (чувствительность к рынку)
        if np.std(recent_market_returns) > 0:
            beta = np.cov(recent_returns, recent_market_returns)[0, 1] / np.var(recent_market_returns)
        else:
            beta = 1.0
        
        # Альфа = средняя доходность - бета * средняя рыночная доходность
        alpha = np.mean(recent_returns) - beta * np.mean(recent_market_returns)
        
        return alpha

# TODO: Реализуйте класс MeanReversionAlpha
class MeanReversionAlpha(Alpha):
    def __init__(self, reversion_threshold: float = 0.02, name: str = 'MeanReversionAlpha'):
        super().__init__(name)
        self.reversion_threshold = reversion_threshold
    
    def calculate_alpha(self, returns: List[float], market_returns: List[float]) -> float:
        """Альфа на основе возврата к среднему: тренд разворачивается"""
        if len(returns) < 10:
            return 0.0
        
        # Расчет отклонения от среднего
        mean_return = np.mean(returns)
        current_return = returns[-1]
        deviation = current_return - mean_return
        
        # Если отклонение значительное, ожидаем возврата к среднему
        if abs(deviation) > self.reversion_threshold:
            # Отрицательная альфа означает, что ожидаем падения (перекупленность)
            # Положительная альфа означает, что ожидаем роста (перепроданность)
            alpha = -deviation * 0.5  # Коэффициент для сглаживания
        else:
            alpha = 0.0
        
        return alpha

# TODO: Реализуйте класс AlphaPortfolio
class AlphaPortfolio(Portfolio):
    def __init__(
        self,
        assets: List[AbstractAsset] = None,
        alpha_strategy: Alpha = None,
        name: str = 'AlphaPortfolio'
    ):
        super().__init__(assets)
        self.alpha_strategy = alpha_strategy or MomentumAlpha()
        self.name = name
        self.portfolio_returns = []
    
    def add_return_data(self, current_prices: Dict[str, float]):
        """Добавление данных о доходности за период"""
        if not self.portfolio_returns:
            # Первое обновление - не можем рассчитать доходность
            for asset in self.assets:
                asset.price = current_prices.get(asset.symbol, asset.price)
            return
        
        # Расчет доходности портфеля
        old_value = self.total_value()
        
        # Обновляем цены активов
        for asset in self.assets:
            new_price = current_prices.get(asset.symbol, asset.price)
            asset.returns_history.append((new_price - asset.price) / asset.price)
            asset.price = new_price
        
        new_value = self.total_value()
        portfolio_return = (new_value - old_value) / old_value if old_value > 0 else 0
        self.portfolio_returns.append(portfolio_return)
    
    def calculate_portfolio_alpha(self, market_returns: List[float]) -> float:
        """Расчет альфы портфеля"""
        if not self.portfolio_returns or not market_returns:
            return 0.0
        
        min_length = min(len(self.portfolio_returns), len(market_returns))
        portfolio_returns_adj = self.portfolio_returns[-min_length:]
        market_returns_adj = market_returns[-min_length:]
        
        return self.alpha_strategy.calculate_alpha(portfolio_returns_adj, market_returns_adj)
    
    def calculate_beta(self, market_returns: List[float]) -> float:
        """Расчет беты портфеля (чувствительность к рынку)"""
        if len(self.portfolio_returns) < 2 or len(market_returns) < 2:
            return 1.0
        
        min_length = min(len(self.portfolio_returns), len(market_returns))
        portfolio_returns = self.portfolio_returns[-min_length:]
        market_returns_adj = market_returns[-min_length:]
        
        if np.std(market_returns_adj) == 0:
            return 1.0
        
        # Бета = ковариация(портфель, рынок) / дисперсия(рынок)
        covariance = np.cov(portfolio_returns, market_returns_adj)[0, 1]
        market_variance = np.var(market_returns_adj)
        
        return covariance / market_variance
    
    def get_risk_metrics(self, market_returns: List[float]) -> Dict[str, float]:
        """Расчет различных метрик риска"""
        if not self.portfolio_returns:
            return {
                'alpha': 0.0,
                'beta': 1.0,
                'sharpe_ratio': 0.0,
                'volatility': 0.0
            }
        
        alpha = self.calculate_portfolio_alpha(market_returns)
        beta = self.calculate_beta(market_returns)
        
        # Волатильность (стандартное отклонение доходности)
        volatility = np.std(self.portfolio_returns) if len(self.portfolio_returns) > 1 else 0.0
        
        # Коэффициент Шарпа (доходность / волатильность)
        mean_return = np.mean(self.portfolio_returns) if self.portfolio_returns else 0.0
        sharpe_ratio = mean_return / volatility if volatility > 0 else 0.0
        
        return {
            'alpha': alpha,
            'beta': beta,
            'sharpe_ratio': sharpe_ratio,
            'volatility': volatility
        }
    
    def __repr__(self):
        return f"AlphaPortfolio(name='{self.name}', assets={len(self.assets)}, strategy={self.alpha_strategy})"

# ===== ДЕМОНСТРАЦИЯ =====
def main():
    print("=== СИСТЕМА АЛЬФА-СТРАТЕГИЙ ===\n")
    
    # Создаем активы
    assets = [
        AbstractAsset("AAPL", 150.0),
        AbstractAsset("GOOGL", 2800.0),
        AbstractAsset("MSFT", 300.0)
    ]
    
    # Создаем рыночные данные (S&P 500-like returns)
    market_returns = [0.02, -0.01, 0.015, 0.005, -0.008, 0.012, 0.018, -0.005, 0.01, 0.007]
    
    print("1. 📊 ТЕСТИРОВАНИЕ MOMENTUM СТРАТЕГИИ:")
    momentum_portfolio = AlphaPortfolio(
        assets=assets.copy(),
        alpha_strategy=MomentumAlpha(lookback_period=5),
        name="Momentum Portfolio"
    )
    
    # Симулируем обновления цен (портфель outperforms рынок)
    price_updates = [
        {"AAPL": 155, "GOOGL": 2850, "MSFT": 310},  # +3.3%
        {"AAPL": 158, "GOOGL": 2900, "MSFT": 315},  # +2.1%
        {"AAPL": 162, "GOOGL": 2950, "MSFT": 320},  # +2.4%
        {"AAPL": 165, "GOOGL": 3000, "MSFT": 325},  # +2.0%
    ]
    
    for prices in price_updates:
        momentum_portfolio.add_return_data(prices)
    
    momentum_metrics = momentum_portfolio.get_risk_metrics(market_returns[:4])
    print(f"   Метрики Momentum: {momentum_metrics}")
    
    print("\n2. 📉 ТЕСТИРОВАНИЕ MEAN REVERSION СТРАТЕГИИ:")
    reversion_portfolio = AlphaPortfolio(
        assets=assets.copy(),
        alpha_strategy=MeanReversionAlpha(reversion_threshold=0.01),
        name="Mean Reversion Portfolio"
    )
    
    # Симулируем волатильные цены (для mean reversion)
    volatile_prices = [
        {"AAPL": 160, "GOOGL": 2900, "MSFT": 320},  # Резкий рост
        {"AAPL": 155, "GOOGL": 2850, "MSFT": 310},  # Коррекция
        {"AAPL": 162, "GOOGL": 2950, "MSFT": 325},  # Рост
        {"AAPL": 158, "GOOGL": 2880, "MSFT": 315},  # Коррекция
    ]
    
    for prices in volatile_prices:
        reversion_portfolio.add_return_data(prices)
    
    reversion_metrics = reversion_portfolio.get_risk_metrics(market_returns[:4])
    print(f"   Метрики Mean Reversion: {reversion_metrics}")
    
    print("\n3. 📈 СРАВНЕНИЕ СТРАТЕГИЙ:")
    print(f"   Momentum Alpha: {momentum_metrics['alpha']:.4f}")
    print(f"   Mean Reversion Alpha: {reversion_metrics['alpha']:.4f}")
    print(f"   Momentum Beta: {momentum_metrics['beta']:.4f}")
    print(f"   Mean Reversion Beta: {reversion_metrics['beta']:.4f}")
    print(f"   Momentum Sharpe: {momentum_metrics['sharpe_ratio']:.4f}")
    print(f"   Mean Reversion Sharpe: {reversion_metrics['sharpe_ratio']:.4f}")
    
    print("\n4. 🔄 ТЕСТИРОВАНИЕ РАЗНЫХ СЦЕНАРИЕВ:")
    
    # Тест с отрицательной альфой (underperforming рынок)
    underperforming_assets = [AbstractAsset("UNDER", 100.0)]
    under_portfolio = AlphaPortfolio(underperforming_assets, MomentumAlpha())
    
    under_prices = [
        {"UNDER": 101},  # +1% vs рынок +2%
        {"UNDER": 101.5}, # +0.5% vs рынок +1.5%
        {"UNDER": 101.8}, # +0.3% vs рынок +1.8%
    ]
    
    for prices in under_prices:
        under_portfolio.add_return_data(prices)
    
    under_metrics = under_portfolio.get_risk_metrics([0.02, 0.015, 0.018])
    print(f"   Underperforming Alpha: {under_metrics['alpha']:.4f} (ожидается отрицательная)")
    
    print("\n5. 🎯 ВЫВОДЫ:")
    if momentum_metrics['alpha'] > 0:
        print("   ✅ Momentum стратегия генерирует положительную альфу")
    else:
        print("   ❌ Momentum стратегия не генерирует альфу")
    
    if abs(reversion_metrics['alpha']) > 0.001:
        print("   ✅ Mean Reversion стратегия активна")
    else:
        print("   ❌ Mean Reversion стратегия неактивна")

if __name__ == "__main__":
    main()

# ПРАКТИЧЕСКОЕ ЗАДАНИЕ 6: Продвинутые техники ООП

# TODO: Реализуйте декоратор для валидации параметров опционов
def validate_option_params(func):
    """
    Декоратор для валидации параметров опционов.
    Проверяет, что price > 0, strike > 0, expiration_date не в прошлом
    """
    def wrapper(self, price, strike, expiration_date, *args, **kwargs):
        # TODO: Добавьте валидацию
        # - price > 0
        # - strike > 0  
        # - expiration_date не в прошлом (используйте datetime)
        pass
    return wrapper

# TODO: Реализуйте класс PortfolioManager
class PortfolioManager:
    """
    Менеджер для управления несколькими портфелями
    """
    def __init__(self, name: str = 'PortfolioManager'):
        # TODO: Реализуйте конструктор
        # Должен содержать словарь портфелей и методы для управления
        pass
    
    def add_portfolio(self, portfolio: Portfolio, name: str = None) -> None:
        # TODO: Добавьте портфель в менеджер
        pass
    
    def remove_portfolio(self, name: str) -> None:
        # TODO: Удалите портфель из менеджера
        pass
    
    def get_portfolio(self, name: str) -> Portfolio:
        # TODO: Получите портфель по имени
        pass
    
    def get_total_value(self) -> float:
        # TODO: Рассчитайте общую стоимость всех портфелей
        pass
    
    def rebalance_portfolios(self, target_weights: Dict[str, float]) -> None:
        # TODO: Ребалансируйте портфели согласно целевым весам
        pass
    
    def __repr__(self) -> str:
        # TODO: Реализуйте красивый вывод менеджера
        pass

# TODO: Реализуйте контекстный менеджер для временного изменения параметров портфеля
class PortfolioContext:
    """
    Контекстный менеджер для временного изменения параметров портфеля
    """
    def __init__(self, portfolio: Portfolio, **kwargs):
        # TODO: Реализуйте конструктор
        # kwargs должны содержать параметры для временного изменения
        pass
    
    def __enter__(self):
        # TODO: Сохраните текущие параметры и примените новые
        pass
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # TODO: Восстановите исходные параметры
        pass

# Пример использования контекстного менеджера:
# with PortfolioContext(portfolio, name='Temporary Portfolio') as temp_portfolio:
#     print(temp_portfolio.name)  # 'Temporary Portfolio'
# print(portfolio.name)  # Исходное имя восстановлено


In [ ]:
from datetime import date, timedelta
from contextlib import contextmanager

# Простой декоратор валидации
def validate_params(func):
    def wrapper(self, price, strike, expiration):
        if price <= 0 or strike <= 0:
            raise ValueError("Цена и страйк должны быть > 0")
        if expiration <= date.today():
            raise ValueError("Дата должна быть в будущем")
        return func(self, price, strike, expiration)
    return wrapper

# Простой менеджер
class SimpleManager:
    def __init__(self):
        self.portfolios = {}
    
    def add(self, name, portfolio):
        self.portfolios[name] = portfolio
    
    def total_value(self):
        return sum(p.total_value() for p in self.portfolios.values())

# Простой контекстный менеджер
@contextmanager
def temp_changes(obj, **changes):
    old_values = {}
    for key, value in changes.items():
        if hasattr(obj, key):
            old_values[key] = getattr(obj, key)
            setattr(obj, key, value)
    try:
        yield obj
    finally:
        for key, old_value in old_values.items():
            setattr(obj, key, old_value)

# Использование
portfolio = Portfolio("Test")
manager = SimpleManager()
manager.add("main", portfolio)

with temp_changes(portfolio, name="Temp"):
    print(portfolio.name)  # Temp
print(portfolio.name)      # Test